In [7]:
import pandas as pd
import numpy as np
import math as m
import re


#imported from import.io -> eventually replace with scrapy
raw = pd.read_csv("Salary data - Raw - Glassdoor.csv")


In [8]:
#deleting useless cols and renaming cols for better readability
if 'Sqlogolink image_alt' in raw: del raw['Sqlogolink image_alt']
if 'Nomargvert link_link' in raw: del raw['Nomargvert link_link']
if 'Small link' in raw: del raw['Small link']
if 'Small link_link' in raw: del raw['Small link_link']

raw.columns = ['source_url', 'image_url', 'company_page', 'role', 'company_name', 'salary_count', 'salary', 'pay_period', 'pay_min','pay_max']
raw.drop_duplicates()

# check for any critical missing values, if so fill or drop from table
raw.apply(lambda x: sum(x.isnull()),axis=0)
# raw['col_name'].fillna(some_value, inplace=True)

source_url        0
image_url       113
company_page    113
role              0
company_name      0
salary_count      0
salary            0
pay_period        0
pay_min           0
pay_max           0
dtype: int64

In [9]:
# Standardization of cols, converting to correct formating.
from currency_converter import CurrencyConverter
c = CurrencyConverter()

def std_role(role):   
    return re.sub('\s\-\s\w*', '', role, flags=re.IGNORECASE)

# removes " (Singapore)" from company names - Glassdoor specific issue
def std_company_name(company_name):   
    return re.sub('\s\(Singapore\)', '', company_name, flags=re.IGNORECASE)

def std_salary_count(salary_count):
    return [int(s) for s in salary_count.split() if s.isdigit()][0]

# "40k-50k" range returns the average(min,max)
def std_salary(salary):
    salary = re.sub('[\,\$]','',salary)   # strips $ and ,
    salary = re.sub('k','000',salary)
    if '-' in salary: 
        min_max_salary = map(int, re.split('\-',salary))
        return int(sum(min_max_salary)/2)
    else: return int(salary)

raw['role'] = raw['role'].apply(std_role)
raw['company_name'] = raw['company_name'].apply(std_company_name)
raw['salary_count'] = raw['salary_count'].apply(std_salary_count)
raw['salary'] = raw['salary'].apply(std_salary)
raw['pay_min'] = raw['pay_min'].apply(std_salary)
raw['pay_max'] = raw['pay_max'].apply(std_salary)

# standardizing the pay period to get monthly and yearly pays
def std_to_year(period, salary, output): 
    if output is 'year': 
        if 'monthly' in period:return salary*12
        else: return salary
    else: 
        if 'monthly' in period: return salary
        else: return int(salary/12)
    
    
raw['yearly'] = raw.apply(lambda row: std_to_year(row['pay_period'], row['salary'],'year'), axis=1)
raw['yearly_min'] = raw.apply(lambda row: std_to_year(row['pay_period'], row['pay_min'],'year'), axis=1)
raw['yearly_max'] = raw.apply(lambda row: std_to_year(row['pay_period'], row['pay_max'],'year'), axis=1)
raw['monthly'] = raw.apply(lambda row: std_to_year(row['pay_period'], row['salary'],'month'), axis=1)
raw['monthly_min'] = raw.apply(lambda row: std_to_year(row['pay_period'], row['pay_min'],'month'), axis=1)
raw['monthly_max'] = raw.apply(lambda row: std_to_year(row['pay_period'], row['pay_max'],'month'), axis=1)

if 'salary' in raw: del raw['salary']
if 'pay_period' in raw: del raw['pay_period']
if 'pay_min' in raw: del raw['pay_min']
if 'pay_max' in raw: del raw['pay_max']

raw = raw[['company_name', 'role', 'salary_count', 'yearly', 'yearly_min', 'yearly_max','monthly','monthly_min','monthly_max','company_page','source_url','image_url']]
raw = raw.sort_values('company_name')
raw

,company_name,role,salary_count,yearly,yearly_min,yearly_max,monthly,monthly_min,monthly_max,company_page,source_url,image_url
183,2359 Media,Software Engineer,2,42000,36000,48000,3500,3000,4000,https://www.glassdoor.com/Salary/2359-Media-Sa...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/377697/2359-m...
45,2359 Media,Senior Software Developer,1,54000,48000,60000,4500,4000,5000,https://www.glassdoor.com/Salary/2359-Media-Sa...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/377697/2359-m...
709,2359 Media,Senior Software Developer,1,54000,48000,60000,4500,4000,5000,https://www.glassdoor.com/Salary/2359-Media-Sa...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/377697/2359-m...
419,2359 Media,Software Engineer,1,52000,50000,54000,4333,4166,4500,https://www.glassdoor.com/Salary/2359-Media-Sa...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/377697/2359-m...
401,3Com,Software Engineer,1,40500,39000,42000,3375,3250,3500,https://www.glassdoor.com/Salary/3Com-Salaries...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/1071/3com-squ...
549,3M,Principal Software Engineer,1,120000,120000,120000,10000,10000,10000,https://www.glassdoor.com/Salary/3M-Salaries-E...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/446/3m-square...
508,701Search,Software Engineer,1,54000,48000,60000,4500,4000,5000,NaN,https://www.glassdoor.com/Salaries/singapore-s...,NaN
781,A*Star,Software Developer Intern,1,9600,9228,9984,800,769,832,https://www.glassdoor.com/Salary/A-Star-Salari...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/254781/a-star...
558,A*Star,Senior Software Analyst,1,60000,60000,60000,5000,5000,5000,https://www.glassdoor.com/Salary/A-Star-Salari...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/254781/a-star...
268,A-IT Software Services,Software,1,108000,108000,108000,9000,9000,9000,https://www.glassdoor.com/Salary/A-IT-Software...,https://www.glassdoor.com/Salaries/singapore-s...,https://media.glassdoor.com/sqls/583171/a-it-s...


In [11]:
# deduplicate same roles from the same company

raw.groupby(level=['company_name', 'role'])
# if company name and role name are same, combine two cols


TypeError: unorderable types: list() > int()

In [ ]:
# Normalization
# normalize the roles etc.

In [ ]:
# bring in the other sources

In [ ]:
# Clustering multiple sources

# cluster the same companies together

Appendix: index of technical roles


Software Engineering		
	Backend Engineer	
	Build/Release Engineer	
	Data Engineer	
	Database Administrator	
	DevOps Engineer	
	Embedded Engineer	
	Engineering Manager	
	Frontend Engineer	
	Full Stack Engineer	
	Machine Learning Engineer	
	Mobile Engineer	
	QA/Test Engineer	
	Security Engineer	
	UX Engineer	
		
Data Science		
	Data Analyst	
	Data Scientist	


-----To be used later-----
Product Management		
	Product Manager	
    
Design		
	Brand/Graphic Designer	
	Product Designer	
	UX Designer	
	UX Researcher	
	Visual/UI Designer	
		
Sales		
	Account Executive	
	Account Manager	
	Business Development	
	Customer Success Manager	
	Sales Development Rep	
	Sales Engineer	
	Sales Manager	
	Sales Operations	
		
Marketing		
	Acquisition Marketing	
	Brand Marketing	
	Community/Events Marketing	
	Content Marketing	
	Lifecycle/Email Marketing	
	Marketing Analytics &amp; Ops	
	Marketing Programs	
	PR / Communications	
	Product Marketing	
		
Recruiting / People Ops		
	HR Business Partner	
	People Ops Management	
	People Ops Specialist	
	Recruiter	
	Recruiting Coordinator	
	Sourcer	
		
		
	Accounting	
	Financial Management / Analysis	
	General Management	
	Office Administration	
		
Other		
		
		
		
		